# Road

- Road Length
- Road Type ('Class')
    - Local (y/n)
    - Major Arterial (y/n)
    - Umimproved (y/n)
    - Ramp (y/n)
    - Freeway (y/n)
    - Other (implied, do not create separate dummy var)
- Construction Status (y/n)

In [1]:
# package imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely

## Road Length

In [2]:
neighborhoods = gpd.read_file('../../data/raw/Neighborhood_Statistical_Areas/Neighborhood_Statistical_Areas.shp')

In [3]:
streets = gpd.read_file('../../data/raw/Road_Centerline/geo_export_c02761a8-1d85-477e-a5cf-01b9f22f4d88.shp')

In [4]:
neighborhoods = neighborhoods.to_crs(epsg=4326)

In [5]:
streets_columns = [
'joinid',
'fromright',
'centerline',
'roadclass',
'fullname',
'toleft',
'toright',
'fullnameab',
'objectid',
'shape_leng',
'fromleft',
'geometry']

In [6]:
streets = streets[streets_columns]

In [7]:
streets = streets.set_index('objectid')

In [8]:
streets_centroids = streets.geometry.centroid

In [9]:
streets_centroids = gpd.GeoDataFrame(streets_centroids)

In [10]:
streets_centroids.columns = ['geometry']

In [11]:
streets_centroids_nhoods = gpd.sjoin(streets_centroids, neighborhoods, how='left')

In [12]:
unmatched_streets = streets_centroids_nhoods[streets_centroids_nhoods['GNOCDC_LAB'].isna()].merge(streets, left_index=True, right_index=True)

In [13]:
unmatched_streets['geometry'] = unmatched_streets.geometry_y

In [14]:
unmatched_streets = gpd.GeoDataFrame(unmatched_streets)

In [15]:
rematched_streets = gpd.sjoin(unmatched_streets[['geometry']], neighborhoods)

In [16]:
rematched_streets['objectid'] = rematched_streets.index

In [17]:
rematched_streets = rematched_streets.dissolve(by='objectid')

In [18]:
rematched_streets = rematched_streets[['OBJECTID', 'GNOCDC_LAB']]

In [19]:
street_nhood_ids = streets_centroids_nhoods.loc[streets_centroids_nhoods.GNOCDC_LAB.notna(), ['OBJECTID', 'GNOCDC_LAB']].append(rematched_streets[['OBJECTID', 'GNOCDC_LAB']])

In [20]:
streets_nhoods = streets.merge(street_nhood_ids, left_index=True, right_index=True)

In [21]:
streets_nhoods.columns = [
    'street_id',
    'fromright',
    'centerline',
    'roadclass',
    'fullname',
    'toleft',
    'toright',
    'fullnameab',
    'shape_leng',
    'fromleft',
    'geometry',
    'nhood_id',
    'nhood']

In [22]:
streets_nhoods['nhood_id'] = streets_nhoods.nhood_id.astype('int')

In [23]:
streets_by_neighborhood = gpd.GeoDataFrame()
for nhood_id in neighborhoods.OBJECTID:
    gdf_nhood = streets_nhoods.loc[streets_nhoods['nhood_id'] == nhood_id, ['street_id', 'roadclass', 'fullname', 'fullnameab', 'geometry', 'nhood_id', 'nhood']]
    streets_by_neighborhood = streets_by_neighborhood.append(gdf_nhood.dissolve(by='street_id'))


In [24]:
streets_by_neighborhood['street_id'] = streets_by_neighborhood.index

In [25]:
streets_by_neighborhood = streets_by_neighborhood.to_crs({'proj': 'lcc',
                                                          'lat_1': 29.3,
                                                          'lat_2': 30.7,
                                                          'lat_0': 28.5,
                                                          'lon_0': -91.33333333333333,
                                                          'x_0': 1000000,
                                                          'y_0': 0,
                                                          'datum': 'NAD83',
                                                          'units': 'us-ft',
                                                          'no_defs': True})

In [26]:
streets_by_neighborhood['street_length_ft'] =  streets_by_neighborhood.geometry.length

In [27]:
streets_by_neighborhood['segment_id'] = streets_by_neighborhood.nhood_id.astype('str') + '_' + streets_by_neighborhood.street_id.astype('str')

## Road Class

In [28]:
streets_by_neighborhood['roadclass'] = streets_by_neighborhood.roadclass.fillna('Other')

In [29]:
streets_by_neighborhood = pd.get_dummies(streets_by_neighborhood, prefix='class', columns=['roadclass'])

In [30]:
streets_by_neighborhood.to_pickle('../../data/interim/features/roads.pickle')

## Construction Status

In [31]:
dpw_projects = gpd.read_file('../../data/raw/Dept_of_Public_Works_Roadwork_Projects/Dept_of_Public_Works_Roadwork_Projects.shp')

dpw_projects['StartDate'] = pd.to_datetime(dpw_projects.StartQtr, errors='coerce')
dpw_projects['StartDate_2'] = pd.to_datetime(dpw_projects[dpw_projects.StartDate.isna()].StartQtr.str.replace('-...', ''))
dpw_projects['StartDate'] = np.where(dpw_projects.StartDate.isna(), dpw_projects.StartDate_2, dpw_projects.StartDate)
dpw_projects['EndDate'] = pd.to_datetime(dpw_projects.EndQtr, errors='coerce')
dpw_projects['EndDate_2'] = pd.to_datetime(dpw_projects[dpw_projects.EndDate.isna()].EndQtr.str.replace('-...', ''))
dpw_projects['EndDate'] = np.where(dpw_projects.EndDate.isna(), dpw_projects.EndDate_2, dpw_projects.EndDate)
dpw_projects = dpw_projects[['OBJECTID', 'PROJID', 'CONSTTYPE', 'PROJTYPE', 'NHOOD', 'geometry', 'StartDate', 'EndDate']]
dpw_projects = dpw_projects.set_index('OBJECTID')

In [32]:
streets_projects = gpd.sjoin(streets_by_neighborhood, dpw_projects)

In [33]:
streets_projects = streets_projects[['segment_id', 'index_right', 'StartDate', 'EndDate']]

In [34]:
streets_projects = streets_projects[streets_projects.EndDate > '2011-12-31']

In [35]:
streets_projects = streets_projects[streets_projects.StartDate < '2018-07-01']

In [36]:
streets_projects.columns = ['segment_id', 'dpw_id', 'StartDate', 'EndDate']

In [37]:
streets_projects.to_pickle('../../data/interim/features/construction_status.pickle')